In [ ]:
!pip install bs4

In [ ]:
!pip install spacy

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
import unicodedata

%matplotlib inline

In [ ]:
# Load Dataset
df=pd.read_csv('blogtext.csv')

In [ ]:
# Drop column
df.drop(columns=['id'],inplace=True)

In [ ]:
# Check certain parameters
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
# Multi labels
df['labels']=df['gender']+','+df['topic']+','+df['sign']+','+df['age'].astype('str')

In [ ]:
# Special Characters
list1=[]
for i in df['text']:
    text = i
    new_text = re.sub(r"[^a-zA-Z0-9 ]", "", text)
    list1.append(new_text)

In [ ]:
# Dropping
df.drop(columns=['gender','age','topic','sign','date'],inplace=True)

In [ ]:
df['TEXT']=list1

In [ ]:
df.drop(columns=['text'],inplace=True)

In [ ]:
# New dataframe
df['labels']=df['labels'].astype('str')

In [ ]:
import ast

In [ ]:
df['labels'].iloc[0]

In [ ]:
# Model Building
X=df['TEXT'].head(80000)
y=df['labels'].head(80000)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=10)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# creat vocabulary and create document-term matrix
countvec=CountVectorizer(stop_words='english',ngram_range=(1, 2),max_df=0.75,min_df=2)
countvec.fit(x_train,y_train)

In [ ]:
# Transformation
x_train_dtm=countvec.transform(x_train)
x_test_dtm=countvec.transform(x_test)

In [ ]:
# Dataframe cant create.
pd.DataFrame(data=x_train_dtm.toarray(),columns=countvec.get_feature_names())

In [ ]:
# Transform Label 
from sklearn.preprocessing import MultiLabelBinarizer
ml=MultiLabelBinarizer()

In [ ]:
y_train.shape

In [ ]:
#Encoding the label

ff=ml.fit_transform(y_train)

In [ ]:
gg=ml.transform(y_test)

In [ ]:
# Dataframe
y_train=pd.DataFrame(data=ff,columns=ml.classes_)
y_test=pd.DataFrame(data=gg,columns=ml.classes_)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

clf=LogisticRegression(solver='lbfgs')
clf=OneVsRestClassifier(clf)

In [ ]:
y_test.shape

In [ ]:
clf.fit(x_train_dtm,y_train)

In [ ]:
# Predict
y_pred=clf.predict(x_test_dtm)

In [ ]:
# Inverse Matrix
ml.inverse_transform(y_pred)[1]

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,jaccard_score,accuracy_score

In [ ]:
# Accuracy Score
accuracy_score(y_test,y_pred)

In [ ]:
# Jaccard score
avg=['micro', 'macro', 'samples', 'weighted']
for i in avg:
    score=jaccard_score(y_test,y_pred,average='samples')
    print(f'score for {i}=',score)

In [ ]:
# Classification report
print(classification_report(y_test,y_pred))

In [ ]:
from textblob import TextBlob

In [ ]:
X

In [ ]:
# Sentiment analysis
list2=[]
for i in df['TEXT']:
    sent=TextBlob(i)
    list2.append(sent.sentiment.polarity)

In [ ]:
df['Sentiment']=list2

In [ ]:
df.head()